<a href="https://colab.research.google.com/github/Aunabil4602/NLP-using-deep-learning/blob/master/fake_news_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
!pip install kaggle

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"amlan107","key":"590556bfeabe7dd9d3f8acc4253a829c"}'}

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [0]:

!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle competitions download -c fake-news

 76% 28.0M/37.0M [00:00<00:00, 39.0MB/s]
100% 37.0M/37.0M [00:00<00:00, 83.7MB/s]
  0% 0.00/40.6k [00:00<?, ?B/s]
100% 40.6k/40.6k [00:00<00:00, 41.0MB/s]
  0% 0.00/9.42M [00:00<?, ?B/s]
100% 9.42M/9.42M [00:00<00:00, 86.6MB/s]


In [7]:
!unzip test.csv.zip

Archive:  test.csv.zip
  inflating: test.csv                


In [8]:
!unzip train.csv.zip

Archive:  train.csv.zip
  inflating: train.csv               


In [0]:
import pandas as pd

In [0]:
testData=pd.read_csv('test.csv')

In [0]:
print(len(testData))

5200


In [0]:
trainData=pd.read_csv('train.csv')

In [0]:
submitData=pd.read_csv('submit.csv')

In [0]:
print(len(trainData))

20800


In [0]:
print(trainData['text'][100])

By Patrick Wood By its very nature, the Internet Corporation for Assigned Names and Numbers (ICANN) is a non-profit organization exclusively run by Technocrats. As... 


In [13]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2020-05-07 05:17:21--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-05-07 05:17:21--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-05-07 05:17:22--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [0]:

import os
import numpy as np
import tensorflow as tf
import numpy as np

In [0]:
vocab_size = 20000
embedding_dim = 50
max_length = 2500
trunc_type='post'
oov_tok = "<OOV>"
title_length=50

In [0]:
embeddings_index = {}
f = open('glove.6B.50d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

num_words = min(vocab_size, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in word_index.items():
    if i >= vocab_size:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [0]:

train_data,test_data=trainData['text'],testData['text']
train_data1,test_data1=trainData['label'],submitData['label']
train_titles=trainData['title']
test_titles=testData['title']


training_sentences = []
training_titles=[]
training_labels = []

ok=True
for it in train_data:
  training_sentences.append(str(it).lower())


for it in train_titles:
  training_titles.append(str(it).lower())

for it in train_data1:
  training_labels.append(np.int32(it))

testing_sentences = []
testing_labels = []
testing_titles=[]

for it in test_data:
  testing_sentences.append(str(it).lower())

for it in test_titles:
  testing_titles.append(str(it).lower())

for it in test_data1:
  testing_labels.append(np.int32(it))
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [0]:
avg=0
bel=0
abv=0
high=0;
mx=0
for it in range(20000):
  temp=len(sequences[it])
  avg+=temp
  if mx<temp:
    mx=temp

  if temp >2000:
    high+=1
  elif (temp) >1000:
    abv+=1
  else :
    bel+=1 
print(avg/20000.0)
print(bel)
print(abv)
print(high)
print(mx)

766.76355
14488
4644
868
24512


In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index 
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type,padding='post')

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [0]:
titles=tokenizer.texts_to_sequences(training_titles)
padded_titles = pad_sequences(titles,maxlen=title_length, truncating=trunc_type,padding='post')

test_titles=tokenizer.texts_to_sequences(testing_titles)
test_padded_titles = pad_sequences(test_titles,maxlen=title_length, truncating=trunc_type,padding='post')


In [40]:
print(padded.shape)
print(padded_titles.shape)
print(padded_titles[2])

(20800, 2500)
(20800, 50)
[ 211    2  738  218  115   35 1464    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]


In [24]:

from keras.initializers import Constant

Using TensorFlow backend.


In [42]:
inputText=tf.keras.layers.Input(shape=(max_length,))        
inputTitle=tf.keras.layers.Input(shape=(title_length,))        

embeddingText=tf.keras.layers.Embedding(vocab_size,embedding_dim,embeddings_initializer=Constant(embedding_matrix),trainable=False)(inputText)
#embeddingModel=tf.keras.layers.Embedding(vocab_size,embedding_dim)(inputShape)
textModel=tf.keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2,return_sequences=True)(embeddingText)
textModel=tf.keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2)(textModel)
textModel=tf.keras.layers.Dense(1, activation='sigmoid')(textModel)

embeddingTitle=tf.keras.layers.Embedding(vocab_size,embedding_dim,embeddings_initializer=Constant(embedding_matrix),trainable=False)(inputTitle)
#embeddingModel=tf.keras.layers.Embedding(vocab_size,embedding_dim)(inputShape)
titleModel=tf.keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2,return_sequences=True)(embeddingTitle)
titleModel=tf.keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2)(titleModel)
titleModel=tf.keras.layers.Dense(1, activation='sigmoid')(titleModel)

avgModel=tf.keras.layers.Average()([textModel,titleModel])

model=tf.keras.models.Model(inputs=[inputText,inputTitle],outputs=avgModel)
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 2500)]       0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            [(None, 50)]         0                                            
__________________________________________________________________________________________________
embedding_13 (Embedding)        (None, 2500, 50)     1000000     input_8[0][0]                    
__________________________________________________________________________________________________
embedding_14 (Embedding)        (None, 50, 50)       1000000     input_9[0][0]                    
____________________________________________________________________________________________

In [0]:
num_epochs=10
model.fit([padded,padded_titles], training_labels_final, epochs=num_epochs, validation_data=([testing_padded,test_padded_titles], testing_labels_final))

Train on 20800 samples, validate on 5200 samples
Epoch 1/10
 1152/20800 [>.............................] - ETA: 1:40:12 - loss: 0.6263 - acc: 0.6398